<a href="https://colab.research.google.com/github/Jeremy-su1/ai-algorithm/blob/main/final/multi_cls_gpt4o_mini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# OpenAI 패키지 설치
!pip install openai

In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import numpy as np
import pandas as pd
import os
from datasets import Dataset, load_dataset, DatasetDict
import ast
import evaluate
from sklearn.metrics import accuracy_score, f1_score, precision_recall_fscore_support
from sklearn.metrics import (accuracy_score,
                             classification_report,
                             confusion_matrix)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
classes = ['Algorithms', 'Backend', 'Data Science', 'Databases', 'Dev Tools', 'Frontend', 'Mobile', 'Systems', 'iOS/macOS']
class2id = {'Algorithms' :0, 'Backend' : 1, 'Data Science' : 2, 'Databases' : 3, 'Dev Tools' : 4, 'Frontend' : 5, 'Mobile' :6, 'Systems' : 7, 'iOS/macOS' : 8}
id2class = {0 : 'Algorithms', 1: 'Backend', 2 : 'Data Science', 3 : 'Databases', 4 : 'Dev Tools', 5 : 'Frontend', 6 : 'Mobile', 7 : 'Systems', 8 :'iOS/macOS'}

In [ ]:
import pandas as pd


valid_df = pd.read_csv("/content/drive/My Drive/AiExpertCource/project/dataset/rev_tag_test_samples.csv")

In [ ]:
len(valid_df)

3000

In [ ]:
selected_columns = ['Title', 'Body', 'Tags_new']  # 원하는 컬럼 이름 리스트
valid_df = valid_df[selected_columns]

In [ ]:
def generate_label(data_point):
  all_labels =  ast.literal_eval(data_point['Tags_new'])
  labels = [0. for i in range(len(classes))]
  for label in all_labels:
      label_id = class2id[label]
      labels[label_id] = 1.
  data_point['labels'] = labels
  return data_point

In [ ]:
len(valid_df)

3000

In [ ]:
valid_df = valid_df.apply(generate_label, axis=1)

In [ ]:
valid_df.head()

,Title,Body,Tags_new,labels
0,Exclude holidays between 2 Dates [Might Be Dup...,"I have searched all over the Internet, and I f...",['Frontend'],"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]"
1,Change image color dynamically,<p>I have a .png or .jpeg image. I want to ch...,['Frontend'],"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]"
2,How to implement full calendar using angular js,<p>I want to add a event calender which is sam...,['Frontend'],"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]"
3,Kotlin - Heart equation,i'm trying to position 100 hundred particles i...,['Mobile'],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]"
4,NumberFormatException error confusion on onlin...,<p>Zybooks keeps throwing this error: </p>\n\n...,['Algorithms'],"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"


In [ ]:
from openai import OpenAI
import os
from getpass import getpass

In [ ]:
os.environ['OPENAI_API_KEY'] =  'aa'
system_input = "You are a classifier that categorizes Stack Overflow posts into 9 labels. Here is the list of labels:  ['Algorithms', 'Backend', 'Data Science', 'Databases', 'Dev Tools', 'Frontend', 'Mobile', 'Systems', 'iOS/macOS'].  Each post typically has 1 label, but some may have up to 2 labels. Provide your answer in the format ### label: "

def call_api(user_input, model="gpt-4o-mini", temperature=0, max_tokens=30):
    try:
        # 메시지 목록을 자동으로 생성해요
        messages = [
            {"role": "system", "content": system_input},
            {"role": "user", "content": user_input}
        ]

        response = OpenAI().chat.completions.create(
            model=model,
            messages=messages,
            temperature=temperature,
            max_tokens=max_tokens  # 최대 토큰 수를 지정해요
        )
        # 생성된 응답을 반환해요
        return response
    except Exception as e:
        return f"Error: {str(e)}"

In [ ]:
counter = [0]

In [ ]:
def predict(test):

  prompt = test["Title"] + " " + test["Body"]
  response = call_api(prompt)

  answer = response.choices[0].message.content.split("### label:")[-1].strip()

  all_labels = [item.strip() for item in answer.split(",")]

  labels = [0. for i in range(len(classes))]
  for label in all_labels:
      # Check if the label is not empty before accessing class2id
      if label:
          try:
              label_id = class2id[label]
              labels[label_id] = 1.0
          except KeyError:
              print(f"Warning: Label '{label}' not found in class2id")

  test['predict'] = labels

  if counter[0] % 100 == 0:
        print(f"Processing row {counter[0]}")
  counter[0] += 1

  return test


In [ ]:
valid_df = valid_df.apply(predict, axis=1)

Processing row 0
Processing row 100
Processing row 200
Processing row 300
Processing row 400
Processing row 500
Processing row 600
Processing row 700
Processing row 800
Processing row 900
Processing row 1000
Processing row 1100
Processing row 1200
Processing row 1300
Processing row 1400
Processing row 1500
Processing row 1600
Processing row 1700
Processing row 1800
Processing row 1900
Processing row 2000
Processing row 2100

To resolve the issue with your program not reading `accountType` as a variable' not found in class2id
Processing row 2200
Processing row 2300
Processing row 2400
Processing row 2500
Processing row 2600
Processing row 2700
Processing row 2800
Processing row 2900


In [ ]:
valid_df.head()

,Title,Body,Tags_new,labels,predict
0,Exclude holidays between 2 Dates [Might Be Dup...,"I have searched all over the Internet, and I f...",['Frontend'],"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]","[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
1,Change image color dynamically,<p>I have a .png or .jpeg image. I want to ch...,['Frontend'],"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]"
2,How to implement full calendar using angular js,<p>I want to add a event calender which is sam...,['Frontend'],"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]"
3,Kotlin - Heart equation,i'm trying to position 100 hundred particles i...,['Mobile'],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]"
4,NumberFormatException error confusion on onlin...,<p>Zybooks keeps throwing this error: </p>\n\n...,['Algorithms'],"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"


In [ ]:
y_pred = np.array([item['predict'] for item in valid_df.to_dict('records')])  # Convert DataFrame to list of dictionaries
y_true = np.array([item['labels'] for item in valid_df.to_dict('records')])  # Convert DataFrame to list of dictionaries

In [ ]:
# define which metrics to compute for evaluation
import evaluate
import numpy as np

clf_metrics = evaluate.combine(["accuracy", "f1", "precision", "recall"])

def sigmoid(x):
   return 1/(1 + np.exp(-x))

def compute_metrics(predictions, labels):

  predictions = sigmoid(predictions)
  predictions = (predictions > 0.5).astype(int)
  accuracy = accuracy_score(labels, predictions)
  precision, recall, f1_score_result, _ = precision_recall_fscore_support(labels, predictions, average='micro')
  precision_macro, recall_macro, f1_score_result_macro, _ = precision_recall_fscore_support(labels, predictions, average='macro')
  precision_weighted, recall_weighted, f1_score_result_weighted, _ = precision_recall_fscore_support(labels, predictions, average='weighted')

  flat_predictions = predictions.reshape(-1)
  flat_labels = labels.reshape(-1)
  flat_accuracy = accuracy_score(flat_labels, flat_predictions)

  return {
        'flat_accuracy' : flat_accuracy,
        'accuracy' : accuracy,
        'micro_precision': precision,
        'micro_recall': recall,
        'micro_f1': f1_score_result,
        'macro_precision': precision_macro,
        'macro_recall': recall_macro,
        'macro_f1': f1_score_result_macro,
        'weighted_precision': precision_weighted,
        'weighted_recall': recall_weighted,
        'weighted_f1': f1_score_result_weighted,
        }

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
print(compute_metrics(y_pred, y_true))

{'flat_accuracy': 0.8551851851851852, 'accuracy': 0.36033333333333334, 'precision': 0.46186186186186184, 'recall': 0.4206783369803063, 'f1_score': 0.4403091898081878}


In [ ]:
valid_df.to_csv('/content/drive/My Drive/AiExpertCource/project/dataset/rev_tag_test_samples_predict.csv', index=False)

In [ ]:
import pandas as pd


valid_df = pd.read_csv("/content/drive/My Drive/AiExpertCource/project/dataset/rev_tag_test_samples_predict.csv")

In [ ]:
valid_df['predict'] = valid_df['predict'].apply(lambda x: np.array(ast.literal_eval(x)))
valid_df['labels'] = valid_df['labels'].apply(lambda x: np.array(ast.literal_eval(x)))

In [ ]:
y_pred = np.array([item['predict'] for item in valid_df.to_dict('records')])  # Convert DataFrame to list of dictionaries
y_true = np.array([item['labels'] for item in valid_df.to_dict('records')])  # Convert DataFrame to list of dictionaries


In [ ]:
print(compute_metrics(y_pred, y_true))

{'flat_accuracy': 0.8551851851851852, 'accuracy': 0.36033333333333334, 'micro_precision': 0.46186186186186184, 'micro_recall': 0.4206783369803063, 'micro_f1': 0.4403091898081878, 'macro_precision': 0.47846134370907695, 'macro_recall': 0.4915343136583551, 'macro_f1': 0.3852802197584824, 'weighted_precision': 0.6405854905982605, 'weighted_recall': 0.4206783369803063, 'weighted_f1': 0.43715144351930046}
